In [ ]:
# Programmatically create and query Tableau Agents using APIs

# Step 1: Run your agent using Langraph dev or Langgraph cloud

In [ ]:
#import required packages
import requests
import json
from IPython.display import Markdown, display

In [ ]:
#generate an instance of a Tableau Data Q&A Agent
def generate_agent(agent_url, graph_id_name):
    url = agent_url + "/assistants"
    
    payload = {
        "assistant_id": "",
        "graph_id": graph_id_name,
        "config": {},
        "metadata": {},
        "if_exists": "raise",
        "name": ""
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=payload, headers=headers)
    
    return(response.json()["assistant_id"])

In [ ]:
my_url = 'http://127.0.0.1:2024'
my_graph_id = 'keynote'
agent_id = generate_agent(my_url, my_graph_id)
agent_id

In [ ]:

endpoint = "/runs/stream"
payload = {
    "assistant_id": agent_id,
    "input": {
        "messages": [
            ("human", "which suppliers do we get the most Bottom Bracket and Display quantities from?")
        ]
    }
}
response = requests.post(my_url+endpoint, json=payload)

In [ ]:
def pretty_print_tab_agent_response(agent_output):
    current_event = None
    for line in agent_output.iter_lines():
        if not line:
            continue
    
        decoded = line.decode("utf-8")
    
        if decoded.startswith("event:"):
            current_event = decoded.replace("event: ", "").strip()
        elif decoded.startswith("data:"):
            data_json = decoded.replace("data: ", "").strip()
    
            try:
                data = json.loads(data_json)
            except json.JSONDecodeError:
                print(f"⚠️ Couldn't parse JSON: {data_json}")
                continue
    
            # 🎯 Final AI response usually in 'values' event
            if current_event == "values":
                messages = data.get("messages", [])
                for msg in messages:
                    if msg.get("type") == "ai" and msg.get("content"):
                        return(display(Markdown(msg["content"])))

pretty_print_tab_agent_response(response)